## phantom change

In [ ]:
import h5py
import io
import glob
import os
import pandas as pd
import numpy as np
from PIL import Image
import copy
import cv2
import math


: 

In [ ]:
# input file path
h5_path = "./phantom.h5"
h5_path_origin = "./phantom_origin.h5"

In [ ]:
with h5py.File(h5_path, "a") as f:
    # check directory structure
    print("Lists of data names : ", f.keys())

    # copy velocity data
    u = f["u"]              # x velocity
    v = f["v"]              # y velocity
    pd = f["pd"]            # proton distribution
    t1 = f["t1"]            # t1
    t2 = f["t2"]            # t2

    # change proton distribution
    # preparation
    nx = 128
    ny = 128
    nz = 1
    pd_0 = np.zeros((128*16, 128*16))
    pitch = 8.0
    x = np.zeros(40)
    y = np.zeros(40)
    
    # calculation
    for i in range(3):
        y[i] =  64*16+2e0*pitch*16*(i-1)
        x[i] = 64*16-pitch*16*math.sqrt(3e0)*2e0
    for i in range(3, 7, 1):
        y[i] = 64*16+2*pitch*16*(i-4.5)
        x[i] = 64*16-pitch*16*math.sqrt(3)*1
    for i in range(7, 12, 1):
        y[i] = 64*16+2*pitch*16*(i-9)
        x[i] = 64*16-pitch*16*math.sqrt(3)*0
    for i in range(12, 16, 1):
        y[i] = 64*16+2*pitch*16*(i-13.5)
        x[i] = 64*16-pitch*16*math.sqrt(3)*(-1)
    for i in range(16, 19, 1):
        y[i] = 64*16 + 2e0*pitch*16*(i-17)
        x[i] = 64*16 - pitch*16*math.sqrt(3e0)*(-2e0)
    
    r1 = pitch*16
    r2 = r1 - 3e0*16
    r1sq = r1*r1
    r2sq = r2*r2
    
    for j in range(ny*16):
        for i in range(nx*16):
            if (i-1024)*(i-1024)+(j-1024)*(j-1024) < 50*16*50*16:
                pd_0[j, i] = 100
            else:
                pd_0[j, i] = 50
                
    for j in range(ny*16):
        for i in range(nx*16):
            for k in range(19):
                a = (j-x[k])*(j-x[k])+(i-y[k])*(i-y[k])
                if a <= r1sq and r2sq<=a:
                    pd_0[j, i] = 50
                    
    for j in range(ny):
        for i in range(nx):
            s0 = 0
            for k in range(16):
                for n in range(16):
                    s0 += pd_0[16*j+k, 16*i+n]
            pd[j, i] = s0 / 256
    
            
